In [64]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
random.seed(2)
qpa = qparray.QpixAsicArray(16,16)

Created ASIC at row 0 col 0 with frq: 55845416.84
Created ASIC at row 0 col 1 with frq: 48342865.23
Created ASIC at row 0 col 2 with frq: 50987149.05
Created ASIC at row 0 col 3 with frq: 50366303.62
Created ASIC at row 0 col 4 with frq: 52087871.45
Created ASIC at row 0 col 5 with frq: 46494725.53
Created ASIC at row 0 col 6 with frq: 48963055.24
Created ASIC at row 0 col 7 with frq: 48121349.62
Created ASIC at row 0 col 8 with frq: 47313418.14
Created ASIC at row 0 col 9 with frq: 47890427.96
Created ASIC at row 0 col 10 with frq: 48718864.04
Created ASIC at row 0 col 11 with frq: 49283015.55
Created ASIC at row 0 col 12 with frq: 47733240.29
Created ASIC at row 0 col 13 with frq: 51055029.91
Created ASIC at row 0 col 14 with frq: 48631224.78
Created ASIC at row 0 col 15 with frq: 42005484.53
Created ASIC at row 1 col 0 with frq: 52976710.89
Created ASIC at row 1 col 1 with frq: 49020292.22
Created ASIC at row 1 col 2 with frq: 48141619.19
Created ASIC at row 1 col 3 with frq: 506709

In [65]:
procQueue = ProcQueue()
stepNum = 0
timeEnd = 2.0
timeNow = 1.0
tickNow = 50e6
deltaT = 1.0
deltaTick = 50e6
timeEpsilon = 1e-6
eventTimes = []
hitsPerEvent = []

while(timeNow < timeEnd):

    for asic in qpa:
      newProcessItems = asic.Process(timeNow - timeEpsilon)
      if len(newProcessItems) > 0:
        print("WARNING: ASIC had things left to do at next major time step")

    procQueue.AddQueueItem(qpa[0][0], 3, PixelHit(tickNow, [], None, None), timeNow)

    while(procQueue.Length() > 0):

        nextItem = procQueue.PopQueue()
        nextAsic = nextItem.asic
        nextTime = nextItem.absTime

        somethingToDo = True
        while somethingToDo:
            somethingToDo = False
            for asic in qpa:
                newProcessItems = asic.Process(nextTime)
                for item in newProcessItems:
                    procQueue.AddQueueItem(*item)
                if len(newProcessItems) > 0:
                    somethingToDo = True

        newQueueItems = nextAsic.ReceiveData(nextItem)
        for item in newQueueItems:
            procQueue.AddQueueItem(*item)

        somethingToDo = True
        while somethingToDo:
            somethingToDo = False
            for asic in qpa:
                newProcessItems = asic.Process(nextTime)
                for item in newProcessItems:
                    procQueue.AddQueueItem(*item)
                if len(newProcessItems) > 0:
                    somethingToDo = True

        stepNum += 1

    eventTimes.append(qpa._daqNode.absTimeNow-timeNow)
    hitsPerEvent.append(qpa._daqNode.daqHits)
    qpa._daqNode.daqHits = 0
    timeNow += deltaT
    tickNow += deltaTick

In [66]:
print("MAX QUEUE DEPTHS")
# Check the current queue depths
for asic in qpa:
    print(str(asic.maxDepth),end='')
    for d in range(0,4):
      print(str(asic.maxConnDepths[d])+" ",end='')
    print()

print("PROCESSING TIMES")
for i in range(0,len(eventTimes)):
  print(str(eventTimes[i]))

print("HITS PER EVENT")
for i in range(0,len(hitsPerEvent)):
  print(str(hitsPerEvent[i]))

MAX QUEUE DEPTHS
30 1 218 0 
20 19 0 0 
30 1 0 0 
20 16 12 0 
20 1 34 0 
20 2 0 0 
50 2 0 0 
10 1 0 0 
40 3 0 0 
50 4 0 0 
20 1 0 0 
20 1 0 0 
50 4 0 0 
30 1 0 0 
20 1 0 0 
50 0 0 0 
20 1 26 0 
50 3 10 0 
30 0 1 0 
10 0 1 0 
10 2 74 0 
10 1 0 0 
40 6 4 0 
30 1 4 0 
20 5 2 0 
10 1 3 0 
20 3 1 0 
40 3 7 0 
30 2 1 0 
30 1 7 0 
20 2 8 0 
40 0 2 0 
20 0 3 0 
20 0 2 0 
20 0 0 0 
30 0 2 0 
10 1 2 0 
40 0 19 0 
40 0 0 0 
40 0 0 0 
20 0 0 0 
40 0 0 0 
10 0 0 0 
20 0 1 0 
10 0 0 0 
20 0 1 0 
20 0 1 0 
40 0 0 0 
20 0 1 0 
20 1 3 0 
30 0 0 0 
20 0 1 0 
30 0 0 0 
30 1 52 0 
20 7 0 0 
40 2 2 0 
30 2 3 0 
20 1 5 0 
40 0 0 0 
20 1 0 0 
30 0 0 0 
50 0 0 0 
20 1 0 0 
40 0 2 0 
20 0 1 0 
50 3 9 0 
30 0 0 0 
20 1 0 0 
30 0 1 0 
30 7 3 0 
20 0 1 0 
20 0 0 0 
30 0 0 0 
20 1 11 0 
10 1 0 0 
30 2 0 0 
20 1 0 0 
30 1 0 0 
20 0 0 0 
40 0 2 0 
10 0 1 0 
20 2 31 0 
10 1 4 0 
20 0 2 0 
20 0 0 0 
30 0 0 0 
10 2 8 0 
30 1 21 0 
20 0 0 0 
30 2 0 0 
40 3 0 0 
60 4 0 0 
20 1 0 0 
30 1 0 0 
10 0 0 0 
20 0 0 0 
30 0 1 0 